In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
# Initialize lemmatizer and load data
lemmatizer = WordNetLemmatizer()
data_file = open(r"D:\\PROJECT\\intents.json").read()
intents = json.loads(data_file)

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']

In [5]:

# Tokenizing and lemmatizing
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = sorted(list(set([lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words])))
classes = sorted(list(set(classes)))

In [6]:
print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

283 documents
129 classes ['What are the types of depression?', 'about', 'afternoon', 'anxious', 'ask', 'at what age does anxiety peak?', 'can lack of sleep make you feel sad?', 'can low blood sugar cause suicidal thoughts?', 'casual', 'creation', 'death', 'default', 'depressed', 'do we control our thoughts?', 'does oversleeping cause depression?', 'done', 'evening', 'fact-1', 'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15', 'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-2', 'fact-20', 'fact-21', 'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27', 'fact-28', 'fact-29', 'fact-3', 'fact-30', 'fact-31', 'fact-32', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9', 'friends', 'goodbye', 'greeting', 'happy', 'hate-me', 'hate-you', 'help', 'how can we reduce anxiety?', 'how does depression affect the world?', 'how long can anxiety last?', 'how many thoughts a day do we have?', 'i am a victim of bullying', 'i am afraid i will fail again', 'i am afraid to file 

In [7]:
pickle.dump(words, open('texts.pkl', 'wb'))
pickle.dump(classes, open('labels.pkl', 'wb'))


In [8]:
pickle.dump(words, open(r'C:\Users\biswa\OneDrive\Desktop\flask\texts.pkl', 'wb'))


In [9]:
words = pickle.load(open(r'C:\Users\biswa\OneDrive\Desktop\flask\texts.pkl', 'rb'))


In [10]:
# Creating training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
    bag = [1 if w in pattern_words else 0 for w in words]
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
train_x = np.array([item[0] for item in training])
train_y = np.array([item[1] for item in training])

print("Training data created")

Training data created


In [11]:
# Model definition and training
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

C:\Users\biswa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


C:\Users\biswa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [13]:
# Fit and save model
hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save('model.h5')
print("Model created and saved.")

Epoch 1/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0085 - loss: 4.8658
Epoch 2/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0164 - loss: 4.7716
Epoch 3/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0542 - loss: 4.6815
Epoch 4/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0461 - loss: 4.6055
Epoch 5/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0776 - loss: 4.4274
Epoch 6/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0693 - loss: 4.3882
Epoch 7/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1474 - loss: 4.1185
Epoch 8/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1317 - loss: 4.1516
Epoch 9/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0729 - loss: 4.0600
Epoch 10/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1626 - loss: 3.7979
Epoch 11/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1347 - loss: 3.7836
Epoch 12/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy:

Model created and saved.


In [14]:
model.save(r'C:\Users\biswa\OneDrive\Desktop\flask\model.h5')


In [ ]:
import nltk
import random
import numpy as np
import json
import pickle
from nltk.stem import WordNetLemmatizer
from keras.models import load_model

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Load the trained model, intents, words, and classes
model = load_model('model.h5')
intents = json.loads(open('D:/PROJECT/intents.json').read())
words = pickle.load(open('texts.pkl', 'rb'))
classes = pickle.load(open('labels.pkl', 'rb'))

# Prediction function
def predict_intent(text):
    # Tokenize and lemmatize the input text
    words_in_text = nltk.word_tokenize(text.lower())
    words_in_text = [lemmatizer.lemmatize(w) for w in words_in_text]
    
    # Create a bag of words based on the model's vocabulary
    bag = [1 if w in words_in_text else 0 for w in words]
    
    # Reshape to match the model's input shape
    bag = np.array(bag).reshape(1, -1)
    
    # Predict the intent using the model
    prediction = model.predict(bag)[0]
    intent_index = np.argmax(prediction)
    
    # If the prediction probability is above 0.5, return the intent, else None
    return classes[intent_index] if prediction[intent_index] > 0.5 else None

# Get response for a given intent
def get_response(intent):
    for i in intents['intents']:
        if i['tag'] == intent:
            return random.choice(i['responses'])
    return "I'm not sure how to respond to that."

# Chat loop
print("Chatbot is ready! Type 'quit' to exit.")
while True:
    user_input = input("You: ")
    
    if user_input.lower() == 'quit':
        print("Exiting... Goodbye!")
        break
    
    intent = predict_intent(user_input)
    
    if intent:
        response = get_response(intent)
    else:
        response = "I don't understand."
    
    print("Chatbot:", response)


Chatbot is ready! Type 'quit' to exit.


You:  hey


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Chatbot: Great to see you. How do you feel currently?


You:  what is sun


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Chatbot: I don't understand.


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Chatbot: It sound like i'm not being very helpful right now.


You:  deth


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Chatbot: I don't understand.


You:  death


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Chatbot: I don't understand.
